![](2023-07-24-10-52-10.png)

In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install langchainhub
!pip install chromadb
!pip install pypdf
!pip install tiktoken
!pip install python-dotenv
!pip install unstructured
!pip install pysqlite3

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

# Building a Simple QA System for Chatting with a PDF

This part of the training will be mostly hands on with the code for building the qa PDF system with langchain.

In [1]:
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain

In [2]:
pdf_path = "./assets-resources/llm_paper_know_dont_know.pdf"
loader = PyPDFLoader(pdf_path) # LOAD

In [3]:
pdf_docs = loader.load_and_split() # SPLIT
pdf_docs

[Document(metadata={'source': './assets-resources/llm_paper_know_dont_know.pdf', 'page': 0}, page_content='Do Large Language Models Know What They Don’t Know?\nZhangyue Yin♢Qiushi Sun♠Qipeng Guo♢\nJiawen Wu♢Xipeng Qiu♢∗Xuanjing Huang♢\n♢School of Computer Science, Fudan University\n♠Department of Mathematics, National University of Singapore\n{yinzy21,jwwu21}@m.fudan.edu.cn qiushisun@u.nus.edu\n{qpguo16,xpqiu,xjhuang}@fudan.edu.cn\nAbstract\nLarge language models (LLMs) have a wealth\nof knowledge that allows them to excel in vari-\nous Natural Language Processing (NLP) tasks.\nCurrent research focuses on enhancing their\nperformance within their existing knowledge.\nDespite their vast knowledge, LLMs are still\nlimited by the amount of information they can\naccommodate and comprehend. Therefore, the\nability to understand their own limitations on\nthe unknows, referred to as self-knowledge,\nis of paramount importance. This study aims\nto evaluate LLMs’ self-knowledge by assess-\ning 

In [4]:
doc_obj = pdf_docs[0]
doc_obj

Document(metadata={'source': './assets-resources/llm_paper_know_dont_know.pdf', 'page': 0}, page_content='Do Large Language Models Know What They Don’t Know?\nZhangyue Yin♢Qiushi Sun♠Qipeng Guo♢\nJiawen Wu♢Xipeng Qiu♢∗Xuanjing Huang♢\n♢School of Computer Science, Fudan University\n♠Department of Mathematics, National University of Singapore\n{yinzy21,jwwu21}@m.fudan.edu.cn qiushisun@u.nus.edu\n{qpguo16,xpqiu,xjhuang}@fudan.edu.cn\nAbstract\nLarge language models (LLMs) have a wealth\nof knowledge that allows them to excel in vari-\nous Natural Language Processing (NLP) tasks.\nCurrent research focuses on enhancing their\nperformance within their existing knowledge.\nDespite their vast knowledge, LLMs are still\nlimited by the amount of information they can\naccommodate and comprehend. Therefore, the\nability to understand their own limitations on\nthe unknows, referred to as self-knowledge,\nis of paramount importance. This study aims\nto evaluate LLMs’ self-knowledge by assess-\ning t

In [5]:
doc_obj.page_content

'Do Large Language Models Know What They Don’t Know?\nZhangyue Yin♢Qiushi Sun♠Qipeng Guo♢\nJiawen Wu♢Xipeng Qiu♢∗Xuanjing Huang♢\n♢School of Computer Science, Fudan University\n♠Department of Mathematics, National University of Singapore\n{yinzy21,jwwu21}@m.fudan.edu.cn qiushisun@u.nus.edu\n{qpguo16,xpqiu,xjhuang}@fudan.edu.cn\nAbstract\nLarge language models (LLMs) have a wealth\nof knowledge that allows them to excel in vari-\nous Natural Language Processing (NLP) tasks.\nCurrent research focuses on enhancing their\nperformance within their existing knowledge.\nDespite their vast knowledge, LLMs are still\nlimited by the amount of information they can\naccommodate and comprehend. Therefore, the\nability to understand their own limitations on\nthe unknows, referred to as self-knowledge,\nis of paramount importance. This study aims\nto evaluate LLMs’ self-knowledge by assess-\ning their ability to identify unanswerable or\nunknowable questions. We introduce an auto-\nmated methodology 

In [6]:
from IPython.display import display, Markdown

Markdown(doc_obj.page_content)

Do Large Language Models Know What They Don’t Know?
Zhangyue Yin♢Qiushi Sun♠Qipeng Guo♢
Jiawen Wu♢Xipeng Qiu♢∗Xuanjing Huang♢
♢School of Computer Science, Fudan University
♠Department of Mathematics, National University of Singapore
{yinzy21,jwwu21}@m.fudan.edu.cn qiushisun@u.nus.edu
{qpguo16,xpqiu,xjhuang}@fudan.edu.cn
Abstract
Large language models (LLMs) have a wealth
of knowledge that allows them to excel in vari-
ous Natural Language Processing (NLP) tasks.
Current research focuses on enhancing their
performance within their existing knowledge.
Despite their vast knowledge, LLMs are still
limited by the amount of information they can
accommodate and comprehend. Therefore, the
ability to understand their own limitations on
the unknows, referred to as self-knowledge,
is of paramount importance. This study aims
to evaluate LLMs’ self-knowledge by assess-
ing their ability to identify unanswerable or
unknowable questions. We introduce an auto-
mated methodology to detect uncertainty in the
responses of these models, providing a novel
measure of their self-knowledge. We further in-
troduce a unique dataset, SelfAware , consisting
of unanswerable questions from five diverse cat-
egories and their answerable counterparts. Our
extensive analysis, involving 20 LLMs includ-
ing GPT-3, InstructGPT, and LLaMA, discov-
ering an intrinsic capacity for self-knowledge
within these models. Moreover, we demon-
strate that in-context learning and instruction
tuning can further enhance this self-knowledge.
Despite this promising insight, our findings also
highlight a considerable gap between the capa-
bilities of these models and human proficiency
in recognizing the limits of their knowledge.
“True wisdom is knowing what you don’t know.”
–Confucius
1 Introduction
Recently, Large Language Models (LLMs) such
as GPT-4 (OpenAI, 2023), PaLM 2 (Anil et al.,
2023), and LLaMA (Touvron et al., 2023) have
shown exceptional performance on a wide range
of NLP tasks, including common sense reason-
ing (Wei et al., 2022; Zhou et al., 2022) and mathe-
∗Corresponding author.
UnknowsKnows UnknowsKnows
Known KnowsKnown Unknows
Unknown Unknows Unknown KnowsUnlockFigure 1: Know-Unknow Quadrant. The horizontal axis
represents the model’s memory capacity for knowledge,
and the vertical axis represents the model’s ability to
comprehend and utilize knowledge.
matical problem-solving (Lewkowycz et al., 2022;
Chen et al., 2022). Despite their ability to learn
from huge amounts of data, LLMs still have lim-
itations in their capacity to retain and understand
information. To ensure responsible usage, it is cru-
cial for LLMs to have the capability of recognizing
their limitations and conveying uncertainty when
responding to unanswerable or unknowable ques-
tions. This acknowledgment of limitations, also
known as “ knowing what you don’t know ,” is a
crucial aspect in determining their practical appli-
cability. In this work, we refer to this ability as
model self-knowledge.
The Know-Unknow quadrant in Figure 1 il-
lustrates the relationship between the model’s
knowledge and comprehension. The ratio of
“Known Knows” to “Unknown Knows” demon-
strates the model’s proficiency in understanding
and applying existing knowledge. Techniques
such as Chain-of-Thought (Wei et al., 2022), Self-
Consistency (Wang et al., 2022), and Complex
CoT (Fu et al., 2022) can be utilized to increasearXiv:2305.18153v2  [cs.CL]  30 May 2023

In [7]:
embeddings = OpenAIEmbeddings() # EMBED
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1315396d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x131549b50>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base='https://api.openai.com/v1', openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [8]:
vectordb = Chroma.from_documents(pdf_docs, embedding=embeddings) # STORE
vectordb

In [9]:
retriever = vectordb.as_retriever() 
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x131586890>)

In [10]:
llm = ChatOpenAI(model="gpt-4o")

In [11]:
llm.invoke("How can I be a better instructor for my live-trainings about AI to get everyone excited about LLMs?")

AIMessage(content='Being an effective and engaging instructor for live trainings about AI, especially on topics like Large Language Models (LLMs), involves a combination of solid content knowledge, interactive teaching techniques, and enthusiasm. Here are some strategies to help you get everyone excited about LLMs:\n\n### 1. **Start with a Hook**\n- **Real-World Examples:** Begin with captivating examples of what LLMs can do, such as generating creative content, aiding in coding, or personalizing user experiences. \n- **Current Events:** Mention recent advancements or newsworthy applications of LLMs to show their relevance and impact.\n\n### 2. **Make It Relatable**\n- **Everyday Applications:** Discuss how LLMs are used in everyday technologies like chatbots, virtual assistants, and recommendation systems.\n- **Personal Stories:** Share personal anecdotes or case studies where LLMs made a significant difference.\n\n### 3. **Simplify Complex Concepts**\n- **Analogies and Metaphors:** U

In [12]:
pdf_qa = RetrievalQA.from_llm(llm=llm, retriever=retriever, return_source_documents=True) # RETRIEVE
pdf_qa
# pdf_qa = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=retriever) # RETRIEVE
# pdf_qa

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x131ca7e10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x16e2cb910>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy='')), document_prompt=PromptTemplate(input_variables=['page_content'], template='Context:\n{page_content}'), document_variable_name='context'), return_source_

In [13]:
query = "Summarize this paper into a set of instructive bullet points."

In [14]:
output = pdf_qa.invoke({"query": query, "chat_history": []}) # adding chat history so the model remembers previous questions

In [15]:
output

{'query': 'Summarize this paper into a set of instructive bullet points.',
 'chat_history': [],
 'result': "- **Objective**: Assess the self-knowledge of language models (LLMs) by evaluating their ability to identify unanswerable questions.\n- **Dataset**: Created the SelfAware dataset, which includes unanswerable questions from sources like Quora and HowStuffWorks, as well as answerable questions from open datasets.\n- **Categories of Unanswerable Questions**:\n  - No scientific consensus\n  - Imagination (speculative future scenarios)\n  - Completely subjective (personal preference)\n  - Too many variables\n  - Philosophical (profound, metaphysical nature)\n- **Reference Sentences**: Selected 16 sentences with uncertain meanings from GPT-3 and InstructGPT for similarity comparison.\n- **Similarity Calculation**: Used SimCSE to compute similarity between generated sentences and reference sentences.\n- **Threshold Ablation**: Determined that a similarity threshold of 0.75 produced the 

In [16]:
final_answer = output["result"]

Markdown(final_answer)

- **Objective**: Assess the self-knowledge of language models (LLMs) by evaluating their ability to identify unanswerable questions.
- **Dataset**: Created the SelfAware dataset, which includes unanswerable questions from sources like Quora and HowStuffWorks, as well as answerable questions from open datasets.
- **Categories of Unanswerable Questions**:
  - No scientific consensus
  - Imagination (speculative future scenarios)
  - Completely subjective (personal preference)
  - Too many variables
  - Philosophical (profound, metaphysical nature)
- **Reference Sentences**: Selected 16 sentences with uncertain meanings from GPT-3 and InstructGPT for similarity comparison.
- **Similarity Calculation**: Used SimCSE to compute similarity between generated sentences and reference sentences.
- **Threshold Ablation**: Determined that a similarity threshold of 0.75 produced the highest F1 score for identifying uncertain sentences.
- **Input Forms**: Evaluated three input forms: Direct, Instruction, and In-Context Learning (ICL).
- **Models Tested**: Included GPT-3, InstructGPT, LLaMA, Alpaca, and Vicuna.
- **Findings**:
  - Instruction tuning significantly enhances model self-knowledge.
  - ICL input form improves self-knowledge by providing richer contextual information.
  - GPT-4 outperforms other models but still lags behind human performance.
- **Human Benchmark**: Volunteers exhibited strong self-knowledge with high F1 scores.
- **Limitations**:
  - Generalization of reference sentences limited to GPT-3 and InstructGPT.
  - Confinement to three input forms.
- **Future Directions**: Propose automated acquisition of more accurate reference sentences and integration of additional cognitive and decision-making methods.
- **Ethical Considerations**: Ensured relevance, harmlessness, privacy, data security, and compliance with dataset licenses.
- **Conclusion**: While LLMs possess some self-knowledge, there's a significant gap compared to humans, indicating the need for further research to enhance LLMs' understanding of their limitations.

In [35]:
query_summary = "What is the main discovery of the paper regarding self-knowledge in LLMs?"

output = pdf_qa.invoke({"query": query_summary, "chat_history": []}) # adding chat history so the model remembers previous questions

In [36]:
output

{'query': 'What is the main discovery of the paper regarding self-knowledge in LLMs?',
 'chat_history': [],
 'result': 'The main discovery of the paper regarding self-knowledge in Large Language Models (LLMs) is that these models inherently possess an intrinsic capacity for self-knowledge. This capacity is demonstrated through their ability to identify unanswerable or unknowable questions to some extent. The paper also finds that techniques such as in-context learning and instruction tuning can further enhance the self-knowledge of LLMs. Despite these promising insights, the study highlights a considerable gap between the capabilities of LLMs and human proficiency in recognizing the limits of their knowledge, with the state-of-the-art model, GPT-4, exhibiting self-knowledge at 75.47% compared to human self-knowledge rated at 84.93%.',
 'source_documents': [Document(page_content='Do Large Language Models Know What They Don’t Know?\nZhangyue Yin♢Qiushi Sun♠Qipeng Guo♢\nJiawen Wu♢Xipeng Q

[Long context issue in LLMs](https://arxiv.org/pdf/2303.18223.pdf)

Long Context. One of the main drawbacks of Transformerbased language models is the context length is limited due to the involved quadratic computational costs in both time
and memory. Meanwhile, there is an increasing demand
for LLM applications with long context windows, such as
in PDF processing and story writing [217]. ChatGPT has
recently released an updated variant with a context window
size of up to 16K tokens, which is much longer than the
initial one, i.e., 4K tokens. Additionally, GPT-4 was launched
with variants with context window of 32K tokens [46]. Next,
we discuss two important factors that support long context
modeling for LLMs.

# References

https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb 
Below are notebook from openai cookbook on these topics of search and embeddings:
- https://github.com/openai/openai-cookbook/blob/main/examples/Get_embeddings.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/Code_search.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/Customizing_embeddings.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/Embedding_Wikipedia_articles_for_search.ipynb
- https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
- [In-context learning abilities of ChatGPT models](https://arxiv.org/pdf/2303.18223.pdf)
- [Issue with long context](https://arxiv.org/pdf/2303.18223.pdf)